<a href="https://colab.research.google.com/github/aqsa-isha/Extract-text-from-images-and-pdfs/blob/main/Extract_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
!pip install pdf2image pytesseract pandas openpyxl
!apt-get install -y poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,002 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 5,002 kB in 2s (3,055 kB/s)
Selecting previously unselected package popp

In [ ]:
import cv2
import numpy as np
from pdf2image import convert_from_path
from pytesseract import image_to_string
import pandas as pd
import re
from google.colab import files

In [ ]:
def preprocess_image(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Use adaptive thresholding instead of a fixed threshold
    binary = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )
    # Resize the image if it's too small for better OCR accuracy
    height, width = binary.shape
    if height < 1000 or width < 1000:
        binary = cv2.resize(binary, (width * 2, height * 2), interpolation=cv2.INTER_CUBIC)
    # Apply morphological operations to enhance text contours
    kernel = np.ones((2, 2), np.uint8)
    morph = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    return morph

In [ ]:
def convert_image_to_text(image):
    preprocessed_image = preprocess_image(image)  # Preprocess the image
    config = "--psm 6"  # Page segmentation mode 6: Assume a single uniform block of text
    text = image_to_string(preprocessed_image, config=config)  # Extract text using OCR
    return text

def clean_text(text):
    return re.sub(r'[^\x20-\x7E\n\r\t]', '', text)

# Upload files
uploaded_files = files.upload()

# Prepare to store extracted text
extracted_data = []

for filename in uploaded_files.keys():
    if filename.endswith(".pdf"):
        # Convert each page of the PDF to an image
        images = convert_from_path(filename)
        pdf_text = ""
        for image in images:
            # Extract text from each page
            extracted_text = convert_image_to_text(image)
            pdf_text += clean_text(extracted_text) + "\n"
        extracted_data.append({"Filename": filename, "Extracted Text": pdf_text})
    else:
        # Process images directly
        img = cv2.imdecode(np.frombuffer(uploaded_files[filename], np.uint8), cv2.IMREAD_COLOR)
        extracted_text = convert_image_to_text(img)
        cleaned_text = clean_text(extracted_text)
        extracted_data.append({"Filename": filename, "Extracted Text": cleaned_text})


Saving 1.jpg to 1.jpg
Saving 2.jpg to 2.jpg
Saving 3.jpg to 3.jpg
Saving 4.jpg to 4.jpg
Saving 5.jpg to 5.jpg
Saving 6.jpg to 6.jpg
Saving 10.jpg to 10.jpg
Saving 11.jpg to 11.jpg
Saving 12.jpg to 12.jpg
Saving 13.jpg to 13.jpg
Saving 14.jpg to 14.jpg
Saving 15.jpg to 15.jpg
Saving 16.jpg to 16.jpg
Saving 17.jpg to 17.jpg
Saving 20.jpg to 20.jpg
Saving 21.jpg to 21.jpg
Saving 22.jpg to 22.jpg
Saving 25.jpg to 25.jpg
Saving 26.jpg to 26.jpg
Saving 27.jpg to 27.jpg
Saving 33.jpg to 33.jpg
Saving 34.jpg to 34.jpg
Saving 35.jpg to 35.jpg
Saving 36.jpg to 36.jpg
Saving 38.jpg to 38.jpg
Saving 41.jpg to 41.jpg
Saving 42.jpg to 42.jpg
Saving 44.jpg to 44.jpg
Saving 45.jpg to 45.jpg
Saving 46.jpg to 46.jpg
Saving 48.jpg to 48.jpg
Saving 49.jpg to 49.jpg
Saving 50.jpg to 50.jpg
Saving 51.jpg to 51.jpg
Saving 52.jpg to 52.jpg
Saving 53.jpg to 53.jpg
Saving 56.jpg to 56.jpg
Saving 61.jpg to 61.jpg
Saving 62.jpg to 62.jpg
Saving 63.jpg to 63.jpg
Saving 64.jpg to 64.jpg
Saving 65.jpg to 65.jpg
Savi

In [ ]:
# Create a DataFrame
df = pd.DataFrame(extracted_data)

# Save to Excel
output_path = "final output.xlsx"
df.to_excel(output_path, index=False)

print(f"Text successfully saved to {output_path}")

Text successfully saved to final output.xlsx
